In [17]:
import pandas as pd

In [18]:
run = "out/2025-04-09_13-12-43_librispeech-pc-test-clean_cs32000_large-v3-turbo_ws15.0_ft6.csv"
df = pd.read_csv(run)
df

,id,wer,pred_transcription,time,pred_transcription_clean,wer_clean
0,2961-961-0000,0.235977,Socrates begins the Timaeus with a summary of ...,105.995751,socrates begins the timaeus with a summary of ...,0.094961
1,4970-29093-0000,0.255000,You'll never dig it out of the Astor Library. ...,121.421180,youll never dig it out of the astor library li...,0.183333
2,6930-76324-0001,0.270833,There were certainly no near the solution of t...,84.058183,there were certainly no near the solution of t...,0.159722
3,7729-102255-0000,0.267874,The bogus legislature numbered 36 members. Thi...,241.035547,the bogus legislature numbered 36 members this...,0.136320
4,5105-28240-0000,0.205128,"Fast as his legs could carry him, Servodak had...",100.095036,fast as his legs could carry him servodak had ...,0.141026
...,...,...,...,...,...,...
78,1995-1837-0000,0.205941,"He knew the silver fleece, his and zora's must...",100.129853,he knew the silver fleece his and zoras must b...,0.061386
79,237-126133-0000,0.321348,"Here she would stay, comforted and soothed amo...",92.567921,here she would stay comforted and soothed amon...,0.184270
80,6829-68771-0000,0.261561,"So, to the surprise of the Democratic Committe...",132.127315,so to the surprise of the democratic committee...,0.114162
81,5683-32879-0000,0.217949,It was not very much past eleven that morning ...,105.007132,it was not very much past eleven that morning ...,0.088172


In [19]:
print("Average WER:", df["wer"].mean())
print("Standard deviation WER:", df["wer"].std())

Average WER: 0.2450971014450712
Standard deviation WER: 0.06308326686011924


In [20]:
def remove_pc(s: str) -> str:
    """Remove the PC from the string."""
    return s.lower().replace(".", "").replace(",", "").replace("'", "").replace('"', "").replace("`", "").replace("(", "").replace(")", "").replace("[", "").replace("]", "").replace("{", "").replace("}", "").replace(";", "").replace(":", "").replace("!", "").replace("?", "")

In [21]:
def load_ground_truth(id: str) -> str:
    """Load the ground truth for the given id."""
    with open(f"data/librispeech-pc-test-clean/{id}/{id}.txt", "r") as f:
        return f.read().strip()

In [22]:
df["true_transcription"] = df["id"].apply(load_ground_truth)
df["true_transcription_clean"] = df["true_transcription"].apply(remove_pc)
df["pred_transcription_clean"] = df["pred_transcription"].apply(remove_pc)

In [23]:
import jiwer

df["wer_clean"] = df.apply(
    lambda row: jiwer.wer(row["true_transcription_clean"], row["pred_transcription_clean"]),
    axis=1,
)

In [24]:
librispeech_total_seconds = 17943.12
total_time = df["time"].sum()

In [27]:
print("Realtime factor", librispeech_total_seconds / total_time)
print("Average WER:", df["wer"].mean())
print("Standard deviation WER:", df["wer"].std())
print("Average WER (no punctuation):", df["wer_clean"].mean())
print("Standard deviation WER (no punctuation):", df["wer_clean"].std())

Realtime factor 1.766933399115167
Average WER: 0.2450971014450712
Standard deviation WER: 0.06308326686011924
Average WER (no punctuation): 0.12923889950858602
Standard deviation WER (no punctuation): 0.052931759551064266


In [26]:
# Remove true transcriptions from output
df = df.drop(columns=["true_transcription", "true_transcription_clean"])
df.to_csv("out/2025-04-09_13-12-43_librispeech-pc-test-clean_cs32000_large-v3-turbo_ws15.0_ft6.csv", index=False)